In [37]:
import pandas as pd
import spacy
from spacy.tokens import Doc
from nltk import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pickle

In [38]:
ner_df = pd.read_csv('../../datasets/extended_df.csv')
ner_df.drop(columns=['Unnamed: 0'], inplace=True)
ner_df['Sentence #'] = ner_df['Sentence #'].str.replace('Sentence: ','')
ner_df['Sentence #'].fillna(method='ffill', inplace=True)
ner_df['Sentence #'] = ner_df['Sentence #'].astype('int64')

In [39]:
sentence_df = ner_df.groupby('Sentence #', as_index=False)['Word'].apply(lambda x:x.str.cat(sep=' '))

In [40]:
nlp = spacy.load('en_core_web_sm')
def sentence_preprocessor(df):
    return_list = []
    for sentence in range(df['Sentence #'].max()):
        words = nlp(Doc(nlp.vocab, df[df['Sentence #'] == sentence + 1].Word.values))
        for word in words:
            # print(word)
            word_base = word
            word_lemma = word.lemma_
            word_pos = word.pos_
            word_tag = word.tag_
            word_dep = word.dep_
            word_length = len(word)
            word_capitalization = str(word)[0].isupper()
            word_punctiation = str(word).isalnum()
            word_stop = word.is_stop
            is_ner = str(word) in set(ent.text for ent in words.ents)
            return_list.append((word_base, word_lemma, word_pos, word_tag, word_dep, word_length, word_capitalization, word_punctiation, word_stop, is_ner))
    return return_list

df_list = sentence_preprocessor(ner_df)
intermediate_df = pd.DataFrame(df_list, columns=['WordBase', 'WordLemma', 'WordPOS', 'WordTag', 
    'WordDep', 'WordLength', 'IsCapitalized', 'NonPunctuation', 'IsStop', 'PossibleNER'])

In [41]:
X_train = intermediate_df[:835700]
X_test = intermediate_df[835700:]
y_train = ner_df.IsNER[:835700]
y_test = ner_df.IsNER[835700:]

In [42]:
xgb_model = XGBClassifier(random_state=42)

categorical_cols = ['WordLemma', 'WordPOS', 'WordTag', 'WordDep']
numerical_cols = ['IsCapitalized', 'NonPunctuation', 'IsStop', 'PossibleNER', 'WordLength']

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('xgb_model', xgb_model)
                            ])

pipeline.fit(X_train, y_train)  

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['IsCapitalized',
                                                   'NonPunctuation', 'IsStop',
                                                   'PossibleNER',
                                                   'WordLength']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['WordLemma', 'WordPOS',
                                                   'WordTag', 'WordDep'])]...
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_threshold=64, max_cat_to_onehot=4,
                               max_delta_step=0, max_depth=6, max_leaves=0,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1, predictor='auto',
                               random_state=42, ...))])

In [43]:
pickle.dump(pipeline, open('ner_model.pkl', 'wb'))

In [44]:
import numpy as np 
import pandas as pd
import string

In [45]:
ner_dataset = pd.read_csv('../../datasets/ner_dataset.csv', 
    encoding='latin1')

In [46]:
ner_dataset['Sentence #'] = ner_dataset['Sentence #'].str.replace('Sentence:', '')
ner_dataset = ner_dataset.fillna(method='ffill')

In [47]:
ner_dataset['Sentence #'] = ner_dataset['Sentence #'].astype(int)

In [48]:
sentences_df = ner_dataset.groupby('Sentence #', as_index=False)['Word'].apply(lambda x: x.str.cat(sep=' '))
sentences_df = sentences_df.rename(columns={'Word': 'Sentences'})

In [49]:
sentences_df.iloc[8411]

Sentence #    8412
Sentences      The
Name: 8411, dtype: object

In [50]:
sentences_df = sentences_df.drop(labels=[8411], axis=0)
sentences_df = sentences_df.reset_index()
sentences_df = sentences_df.drop(columns='index')

In [51]:
import nltk
from nltk.corpus import stopwords 
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [52]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/cody/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/cody/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/cody/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cody/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/cody/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
def _lemmatize_words(sentence):
    wordnet_map = {'N':wordnet.NOUN, 'V':wordnet.VERB, 'J':wordnet.ADJ, 'R':wordnet.ADV}
    pos_tagged_text = nltk.pos_tag(sentence.split())
    return ' '.join([WordNetLemmatizer().lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN))
                    for word, pos in pos_tagged_text])
    
def lda_sent_process(text):
    text = text.lower()  
    PUNCT_TO_REMOVE = string.punctuation
    text = text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    STOPWORDS = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in STOPWORDS])
    text = _lemmatize_words(text)
    return text.split()

In [54]:
sentences_df['lda_sents'] = sentences_df['Sentences'].apply(lambda x: lda_sent_process(x))

In [55]:
from gensim.corpora.dictionary import Dictionary 
from gensim import models 
import re

In [56]:
dct = Dictionary(sentences_df['lda_sents'])

In [57]:
corpus = [dct.doc2bow(sentence) for sentence in sentences_df['lda_sents']]
lda = models.LdaModel(corpus, num_topics=15, random_state=36)

In [58]:
topics = lda.print_topics()

In [59]:
for topic in topics:
  key_indices = re.findall(r'"(.*?)"', topic[1])
  key_words = [dct[int(idx)] for idx in key_indices]
  print(f'Topic {topic[0]}: ', key_words)

Topic 0:  ['vote', 'bird', 'flu', 'last', 'say', 'month', 'election', 'since', 'confirm', 'case']
Topic 1:  ['minister', 'say', 'prime', 'mr', 'president', 'official', 'government', 'foreign', 'chavez', 'tuesday']
Topic 2:  ['foreign', 'east', 'beijing', 'france', 'britain', 'russia', 'gas', 'russian', 'french', 'august']
Topic 3:  ['say', 'kill', 'attack', 'u', 'force', 'military', 'official', 'bomb', 'afghanistan', 'iraqi']
Topic 4:  ['say', 'iran', 'united', 'state', 'nuclear', 'program', 'country', 'nation', 'international', 'un']
Topic 5:  ['say', 'official', 'police', 'news', 'arrest', 'city', 'report', 'force', 'spokesman', 'muslim']
Topic 6:  ['party', 'election', 'new', 'opposition', 'president', 'rule', 'mr', 'political', 'leader', 'call']
Topic 7:  ['say', 'woman', 'result', 'kilometer', 'saudi', 'see', 'make', 'get', 'show', 'heavy']
Topic 8:  ['people', 'say', 'kill', 'least', 'israeli', 'two', 'militant', 'attack', 'death', 'official']
Topic 9:  ['world', 'year', 'high', 

In [60]:
topics_dict = {}
for num, topics in enumerate(topics):
  key_indices = re.findall(r'"(.*?)"', topic[1])
  key_words = [dct[int(idx)] for idx in key_indices]
  topics_dict[num] = key_words

In [61]:
topics_dict = {
  0: ['vote', 'bird', 'election', 'flu'], 
  1: ['minister', 'prime', 'north', 'south', 'president', 'korea'],
  2: ['foreign', 'Beijing', 'Britain', 'France', 'gas', 'German', 'Middle', 'East', 'Russian'], 
  3: ['kill', 'attack', 'military', 'bomb', 'force'],
  4: ['Iran', 'United', 'State', 'nuclear', 'European', 'international'],
  5: [ 'police', 'force', 'city', 'Muslim', 'spokesman', 'security' ], 
  6: ['party', 'president', 'election', 'leader'], 
  7: ['woman', 'citizen', 'explosive'],
  8: ['Israeli', 'death', 'kill', 'Islamic', 'militant'],
  9: ['world', 'economic', 'economy', 'price', 'country'],
  10: ['Lebanon', 'blast', 'responsibility', 'explosion', 'group'],
  11: ['government', 'president', 'Israel', 'Palestinian','peace', 'leader'], 
  12: ['United', 'State', 'secretary', 'storm', 'hurricane', 'emergency'],
  13: ['president', 'charge', 'right', 'court', 'Iraq', 'house'],
  14: ['oil', 'company',  'market', 'demand', 'production', 'decline', 'power', 'government'],
  }

In [62]:
def get_topics(new_text, lda_model, dct): 
  '''
  new_text: str
  lda_model: load from lda.pkl
  dct: load from dct.pkl
  '''
  new_text_doc = lda_sent_process(new_text)
  topics = lda_model[dct.doc2bow(new_text_doc)]
  for num, topic in enumerate(topics): 
    print(f'Topic {topic[0]}:  with probability {topic[1]}')
    print(topics_dict[num])

In [63]:
with open('dct.pkl', 'wb') as pickle_dict: 
  pickle.dump(dct, pickle_dict)
with open('lda.pkl', 'wb') as pickle_lda:
  pickle.dump(lda, pickle_lda)

In [64]:
!pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [65]:
from sentence_transformers import SentenceTransformer

In [66]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [67]:
embeddings = model.encode(sentences_df['Sentences'])

In [68]:
from sklearn.neighbors import NearestNeighbors 
nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(embeddings)

In [69]:
def get_near_sent(text, emb_model, knn_model):
  embedding = emb_model.encode([text])
  _, index = knn_model.kneighbors(embedding)
  for idx in range(index.shape[1]):
    print(f'{idx+1}.',  sentences_df['Sentences'][index[0,idx]])

In [70]:
get_near_sent(sentences_df['Sentences'][0], emb_model=model, knn_model=nbrs)

1. Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
2. Thousands of anti-war protesters have marched in London joining protests in Japan , Australia and elsewhere in the world ahead of the third anniversary of the U.S.-led invasion of Iraq .
3. Thousands of people in cities across Britain have rallied to protest Israeli military action in Lebanon .


In [71]:
with open('emb_model.pkl', 'wb') as pickle_emb:
  pickle.dump(model, pickle_emb)
with open('knn_modle.pkl', 'wb') as pickle_knn:
  pickle.dump(nbrs, pickle_knn)